In [1]:
import psycopg2
from radiospectra.sources import CallistoSpectrogram
import numpy as np
import config as ecallisto_config
import pandas as pd

/tmp/pycharm_project_368/validation/rating_system/radiospectra/version.py:12: UserWarning: could not determine radiospectra package version; this indicates a broken installation
  warnings.warn(


In [2]:
def _get_db():
        """
        Connects to the database and returns a database connection object.

        Returns:
            psycopg2.extensions.connection: The database connection object.
        """
        database = psycopg2.connect(
            host=ecallisto_config.DB_HOST,
            user=ecallisto_config.DB_USER,
            database=ecallisto_config.DB_DATABASE,
            port=ecallisto_config.DB_PORT,
            password=ecallisto_config.DB_PASSWORD
        )
        return database


def convert_to_stars(score):
    ranking = [1.0, 2.0, 3.0, 4.0, 5.0]

    # Compute the percentile of the data
    percent = np.percentile(score, [0, 25, 50, 75, 100])
    rating = np.interp(score, percent, ranking)
    rating = np.round(rating, 2)

    return rating


def _get_all_instruments(start_date, end_date):
    """
    Retrieves instrument data from the database between the given start_date and end_date.

    Args:
        start_date (str): Start date in the format 'YYYY-MM-DD HH:MM:SS'.
        end_date (str): End date in the format 'YYYY-MM-DD HH:MM:SS'.

    Returns:
        list: List of instrument data.
    """
    database = _get_db()

    sql_query_instruments = """
    SELECT *, lower(observation_times) AS start_date, upper(observation_times) AS end_date
    FROM test
    WHERE observation_times && tsrange(%s, %s) AND snr IS NOT NULL
    ORDER BY snr DESC;
        """

    with database.cursor() as cursor:
        cursor.execute(sql_query_instruments, (start_date, end_date))
        instruments = cursor.fetchall()

    return instruments



def get_station_files(start_date, end_date, station_name):
    rows = _get_all_instruments(start_date, end_date)
    result = []

    for row in rows:
        spec = CallistoSpectrogram.read(ecallisto_config.DATA_PATH + row[6])
        file_station_name = spec.header['INSTRUME']

        if file_station_name == station_name:
            file_name = row[6]
            std = row[4]
            snr = row[5]
            result.append((file_name, file_station_name, std, snr))

    columns = ['File Name', 'Station Name','SNR', 'Std']



    df = pd.DataFrame(result, columns=columns)

    df['snr_rating'] = convert_to_stars(df['SNR'])
    df['std_rating'] = convert_to_stars(df['Std'])
    
    

    return df


def get_available_stations_with_averages(start_date, end_date):
    rows = _get_all_instruments(start_date, end_date)
    result = []

    for row in rows:
        spec = CallistoSpectrogram.read(ecallisto_config.DATA_PATH + row[6])
        file_station_name = spec.header['INSTRUME']
        file_name = row[6]
        std = row[4]
        snr = row[5]
        obs_start = row[-2]
        obs_end = row[-1]
        duration = (obs_end - obs_start).total_seconds()
        result.append((file_station_name, std, snr, duration))

    columns = ['Station Name', 'Std', 'SNR', 'Duration']

    df = pd.DataFrame(result, columns=columns)

    # Group by station name and calculate the mean of Std and SNR
    df_grouped = df.groupby('Station Name').agg({
        'Std': 'mean',
        'SNR': 'mean',
        'Duration': 'sum'
    }).reset_index()

    # Convert the SNR and Std to star ratings
    df_grouped['snr_rating'] = convert_to_stars(df_grouped['SNR'])
    df_grouped['std_rating'] = convert_to_stars(df_grouped['Std'])

    # Convert duration to a human-readable format
    df_grouped['Duration'] = pd.to_timedelta(df_grouped['Duration'], unit='s')
    df_grouped['Duration'] = df_grouped['Duration'].apply(
        lambda duration: f"{duration.components.days} days, {duration.components.hours} hours, {duration.components.minutes} minutes, {duration.components.seconds} seconds"
    )

    # Sort by SNR in descending order
    df_grouped = df_grouped.sort_values(by='SNR', ascending=False)

    return df_grouped

start_date = '2020-10-16 07:00:03'
end_date = '2020-10-20 14:30:03'

df_stations = get_available_stations_with_averages(start_date, end_date)

In [3]:
df_stations

,Station Name,Std,SNR,Duration,snr_rating,std_rating
29,MRT3,0.353214,1.230496,"2 days, 7 hours, 54 minutes, 19 seconds",5.00,1.30
33,OOTY,0.376093,1.113477,"3 days, 12 hours, 20 minutes, 0 seconds",4.67,1.35
27,MRT1,0.442425,1.075425,"2 days, 0 hours, 52 minutes, 25 seconds",4.56,1.50
28,MRT2,0.416559,1.031231,"2 days, 13 hours, 44 minutes, 46 seconds",4.43,1.44
34,ROSWELL-NM,0.721055,0.992532,"12 days, 23 hours, 0 minutes, 0 seconds",4.32,2.26
7,AUSTRIA-OE3FLB,0.992565,0.946051,"4 days, 2 hours, 43 minutes, 39 seconds",4.19,3.39
39,SWISS-HB9SCT,0.766350,0.943908,"4 days, 1 hours, 32 minutes, 41 seconds",4.18,2.48
13,GREENLAND,1.016178,0.940780,"8 days, 15 hours, 30 minutes, 0 seconds",4.17,3.47
1,ALASKA-COHOE,0.544381,0.927274,"5 days, 19 hours, 44 minutes, 40 seconds",4.13,1.73
4,ALMATY,0.765500,0.918931,"3 days, 0 hours, 29 minutes, 8 seconds",4.11,2.47


In [1]:
start_date = '2022-03-08 14:30:03'
end_date = '2022-03-09 14:30:03'
station_name = 'INDONESIA'
df = get_station_files(start_date, end_date, station_name)

NameError: name 'get_station_files' is not defined